<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/CSC645/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import numpy as np
#import cupy as np
from keras.utils import to_categorical



## The data

THe movie review dataset is a set of 50000 reviews of movies (half training, half test). Each review contains a set of words and is labeled positive (1) or negative (0). For convenience each word index refer to its frequency of occurence in the dataset. For example a word with index 5 is the fifth most frequently used data set. The indices 0,1 and 2 are reserved so 5 really means the third most frequent.

Details about the dataset can be found here [Keras IMDB](https://keras.io/api/datasets/imdb/)

In this exercise we choose only the first 10000 most frequent words to be included. Any word that is not among them is given the index 2.

First we load data set without omitting any words

### Data details

We would like to have an idea about the number of reviews, the average length of a review. Also we compute how many entries with values 0,1,2 and 3. The number 0 is used for padding and 1 to denote the beginning of each sequence. The number 2 is used for missing words. Finally, the number 3 is never used since as you will see later we will shift the indices by 3.

In [5]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.imdb.load_data()

print("The number of reviews in the x_train data set = {}\n".format(x_train.shape[0]))
print("The average length of reviews = {}".format(np.mean([len(x) for x in x_train])))
print("With standard deviation = {}".format(np.std([len(x) for x in x_train])))
print("The number of 0's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==0])))
print("The number of 1's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==1])))
print("The number of 2's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==2])))
print("The number of 3's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==3])))

The number of reviews in the x_train data set = 25000

The average length of reviews = 238.71364
With standard deviation = 176.49367364852034
The number of 0's in the x_train data set = 0

The number of 1's in the x_train data set = 25000

The number of 2's in the x_train data set = 1

The number of 3's in the x_train data set = 0



Now when we choose only the first 5000 most frequent words and compute the number of 2's in the data set. As you can see the number of 2's is now very large since all the "ignored" words were given the code 2.

In [6]:
max_words=5000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_words)
print("The number of 2's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==2])))

The number of 2's in the x_train data set = 592372



### Word index

Keras provides also a dictionary of word to index. From that we build a dictionary of index to words. We use the index_to_word to display the first review in the data set.

In [7]:

word_to_index=imdb.get_word_index()
print(word_to_index['the'])
print(word_to_index['and'])
index_to_word=dict([(key,val) for (val,key) in word_to_index.items()])
review = " ".join( [index_to_word.get(i - 3, "***") for i in x_train[0]] )



def vectorize(sequences, dimension = max_words):
 results = np.zeros((len(sequences), dimension))
 for i, sequence in enumerate(sequences):
  results[i, sequence] = 1
 return results

x_train=vectorize(x_train)
x_test=vectorize(x_test)






1646592/1641221 [==============================] - 0s 0us/step
1
2


In [8]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1,input_shape=(x_train.shape[1],),activation="sigmoid"))
model.summary()
model.compile(optimizer="Adam",loss="binary_crossentropy",
    metrics=["accuracy"])
history = model.fit(
    x_train,
    y_train,
    batch_size=500,
    epochs=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5001      
Total params: 5,001
Trainable params: 5,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
50/50 [==============================] - 0s 5ms/step - loss: 0.6100 - accuracy: 0.7289
Epoch 2/2
50/50 [==============================] - 0s 5ms/step - loss: 0.4914 - accuracy: 0.8407


In [9]:
x_train=x_train.T
x_test=x_test.T

In [10]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

In [11]:
def initialize_with_zeros(dim):
    
#w is a column vector. later on we take the transpose before operating on w
    w = np.zeros((1,dim))
    b = 0
    
    return w, b

In [12]:
def propagate(w, b, X, Y):
# m is the number of samples which is the number of columns in X
    m = X.shape[1]
    Y_hat= sigmoid(np.dot(w,X)+b)   # compute activation
   
    cost=-np.sum(Y*np.log(Y_hat)+(1-Y)*np.log(1-Y_hat))/m
    # Compute Derivatives

    dw = np.dot(X,(Y_hat-Y).T)/m
    db = np.sum(Y_hat-Y)/m
    cost = np.squeeze(cost)
    
    return dw, db, cost


In [13]:
def learn(w, b, X, Y, num_iterations,learning_rate, print_cost = False):
    
    for i in range(num_iterations):
        # Cost and gradient calculation 
        dw,db, cost = propagate(w,b,X,Y)
        
        # update rule
        w = w-learning_rate*dw.T
        b = b-learning_rate*db

        if print_cost and i % 500 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    
    return w,b,dw,db

In [14]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    
    # Compute vector "Y_hat" predicting
    #    the probabilities of a ship being present in the picture

    Y_hat = sigmoid(np.dot(w,X)+b)

    for i in range(Y_hat.shape[1]):
        
        # Convert probabilities Y_hat[0,i] to actual predictions p[0,i]
        if Y_hat[0,i]>=0.5:
            Y_prediction[0,i]=1
        else:
            Y_prediction[0,i]=0
    
    assert(Y_prediction.shape == (1, m))

    return Y_prediction

In [15]:
# initialize parameters with zeros 
w, b = initialize_with_zeros(x_train.shape[0])



In [16]:
num_iterations = 2000
learning_rate = 0.2
print_cost = True


    


# Gradient descent 
w,b,dw,db= learn(w, b, x_train, y_train, num_iterations, learning_rate, print_cost)


# Predict test/train set examples
Y_prediction_test = predict(w, b, x_test)
Y_prediction_train = predict(w, b, x_train)

# Print train/test Errors


print("train accuracy:"+str((100 - np.mean(np.abs(Y_prediction_train - y_train)) * 100)))
print("test accuracy:"+str((100 - np.mean(np.abs(Y_prediction_test - y_test)) * 100)))

Cost after iteration 0: 0.693147
Cost after iteration 500: 0.302174
Cost after iteration 1000: 0.266001
Cost after iteration 1500: 0.247101
train accuracy:91.428
test accuracy:88.3
